## Data Collection Notebook for Approval Predict

## Objectives

Fetch data from Kaggle and save as raw data.
Inspect the data and save it under outputs/datasets/collection.


## Inputs
Kaggle authentication token (JSON file).

## Outputs
Generate dataset with an output of the loan_approval.csv

## Additional Comments

Loan Approval Dataset is a synthetic dataset with 8 columns relevant to loan approval.

The data was published on Kaggle by user Anish Dev Edward. The dataset contains information used to predict whether a loan application will be approved or rejected, based on applicant and financial details.

License: MIT

Fetch data from Kaggle

In [1]:
import os
import sys

from src.utility.project_setup import setup_project_path

setup_project_path()

Changed to project root directory


'/workspaces/Approval_Predict'

In [2]:
from pathlib import Path
import pandas as pd

root = setup_project_path()
file_path = Path(root) / "outputs" / "datasets" / "collection" / "loan_approval.csv"

if not file_path.exists():
    raise FileNotFoundError(f"Dataset not found at: {file_path}")

df = pd.read_csv(file_path).drop(['name'], axis=1)
df.head(3)

No 'jupyter_notebooks' directory found the the current path


,city,income,credit_score,loan_amount,years_employed,points,loan_approved
0,East Jill,113810,389,39698,27,50.0,0
1,New Jamesside,44592,729,15446,28,55.0,0
2,Lake Roberto,33278,584,11189,13,45.0,0


In [ ]:
%pip install kaggle==1.5.12

In [4]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

chmod: cannot access 'kaggle.json': No such file or directory


In [3]:
KaggleDatasetPath = "anishdevedward/loan-approval-dataset/data?select=loan_approval.csv"
DestinationFolder = "inputs/datasets/raw"   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

Traceback (most recent call last):
  File "/home/cistudent/.local/bin/kaggle", line 3, in <module>
    from kaggle.cli import main
  File "/home/cistudent/.local/lib/python3.12/site-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/home/cistudent/.local/lib/python3.12/site-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /home/cistudent/.kaggle. Or use the environment method.


In [4]:
! unzip -o {DestinationFolder}/*.zip -d {DestinationFolder} \
    && rm {DestinationFolder}/*.zip \
    && rm kaggle.json

unzip:  cannot find or open inputs/datasets/raw/*.zip, inputs/datasets/raw/*.zip.zip or inputs/datasets/raw/*.zip.ZIP.

No zipfiles found.


The dataset has 8 columns: name,city,income,credit_score,loan_amount,years_employed,points,loan_approved

In [5]:
import pandas as pd
df = pd.read_csv(f"inputs/datasets/raw/loan_approval.csv")
df.head()

,name,city,income,credit_score,loan_amount,years_employed,points,loan_approved
0,Allison Hill,East Jill,113810,389,39698,27,50.0,False
1,Brandon Hall,New Jamesside,44592,729,15446,28,55.0,False
2,Rhonda Smith,Lake Roberto,33278,584,11189,13,45.0,False
3,Gabrielle Davis,West Melanieview,127196,344,48823,29,50.0,False
4,Valerie Gray,Mariastad,66048,496,47174,4,25.0,False


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            2000 non-null   object 
 1   city            2000 non-null   object 
 2   income          2000 non-null   int64  
 3   credit_score    2000 non-null   int64  
 4   loan_amount     2000 non-null   int64  
 5   years_employed  2000 non-null   int64  
 6   points          2000 non-null   float64
 7   loan_approved   2000 non-null   bool   
dtypes: bool(1), float64(1), int64(4), object(2)
memory usage: 111.5+ KB


In [7]:
df[df.duplicated(subset=['name'])]

,name,city,income,credit_score,loan_amount,years_employed,points,loan_approved


In [8]:
df['loan_approved'].unique()

array([False,  True])

In [9]:
df['loan_approved'] = df['loan_approved'].replace({"True":1, "False":0})
df['loan_approved'].dtype
df['loan_approved'] = df['loan_approved'].astype(int)
df['loan_approved'].dtype

dtype('int64')

Files saved below in output folder

In [10]:
import os
try:
  os.makedirs(name='outputs/datasets/collection')
except Exception as e:
  print(e)

df.to_csv(f"outputs/datasets/collection/loan_approval.csv",index=False)

[Errno 17] File exists: 'outputs/datasets/collection'
